# Data Access

In [1]:
import pandas as pd

## Accessing Series Elements

In the previous sections, You’ve seen how a Series object is similar to lists and dictionaries in several ways. A further similarity is that you can use the **indexing operator** (`[]`) for Series as well.

You’ll also learn how to use two Pandas-specific access methods:

- `.loc`
- `.iloc`

You’ll see that these data access methods can be much more readable than the indexing operator.

### Using the Indexing Operator

Recall that a Series has two indices:

- A positional or implicit index, which is always a `RangeIndex`
- A label or explicit index, which can contain any hashable objects

In [3]:
city_revenues = pd.Series(
    [4200, 8000, 6500],
    index=["Amsterdam", "Toronto", "Tokyo"]
)
city_revenues

Amsterdam    4200
Toronto      8000
Tokyo        6500
dtype: int64

You can conveniently access the values in a `Series` with both the label and positional indices:

In [6]:
city_revenues["Toronto"]

8000

In [7]:
city_revenues[1]

8000

You can also use negative indices and slices, just like you would for a list:

In [8]:
city_revenues[-1]

6500

In [9]:
city_revenues[1:]

Toronto    8000
Tokyo      6500
dtype: int64

In [10]:
city_revenues['Toronto':]

Toronto    8000
Tokyo      6500
dtype: int64

In [12]:
city_revenues[['Toronto', 'Tokyo']]

Toronto    8000
Tokyo      6500
dtype: int64

### Using `.loc` and `.iloc`

The indexing operator (`[]`) is convenient, but there’s a caveat. What if the labels are also numbers? Say you have to work with a `Series` object like this:

In [4]:
colors = pd.Series(
    ["red", "purple", "blue", "green", "yellow"],
    index=[1, 2, 3, 5, 8]
)
colors

1       red
2    purple
3      blue
5     green
8    yellow
dtype: object

What will `colors[1]` return? For a positional index, `colors[1]` is `"purple"`. However, if you go by the label index, then `colors[1]` is referring to `"red"`.

In [7]:
colors[1]

'red'

The good news is, you don’t have to figure it out! Instead, to avoid confusion, the Pandas Python library provides two data access methods:

- `.loc` refers to the **label index**.
- `.iloc` refers to the **positional index**.

These data access methods are much more readable:

In [8]:
colors.loc[1]

'red'

In [9]:
colors.iloc[1]

'purple'

`colors.loc[1]` returned `"red"`, the element with the label `1`. `colors.iloc[1]` returned `"purple"`, the element with the index `1`.

The following figure shows which elements `.loc` and `.iloc` refer to:

<img src="../images/loc-iloc.png" alt="loc-iloc" width=400 align="left" />

Again, `.loc` points to the **label index** on the right-hand side of the image. Meanwhile, `.iloc` points to the **positional index** on the left-hand side of the picture.

It’s easier to keep in mind the distinction between `.loc` and `.iloc` than it is to figure out what the indexing operator will return. Even if you’re familiar with all the quirks of the indexing operator, it can be dangerous to assume that everybody who reads your code has internalized those rules as well!

> **Note:** In addition to being confusing for Series with numeric labels, the Python indexing operator has some **performance drawbacks**. It’s perfectly okay to use it in interactive sessions for ad-hoc analysis, but for production code, the `.loc` and `.iloc` data access methods are preferable. For further details, check out the Pandas User Guide section on [indexing and selecting data](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html).

`.loc` and `.iloc` also support the features you would expect from indexing operators, like slicing. However, these data access methods have an important difference. While `.iloc` **excludes** the closing element, `.loc` **includes** it. Take a look at this code block:

In [12]:
# Return the elements with the implicit index: 1, 2
colors.iloc[1:3]

2    purple
3      blue
dtype: object

In [15]:
my_list = [1,2,3,4]
my_list[1:3]

[2, 3]

If you compare this code with the image above, then you can see that `colors.iloc[1:3]` returns the elements with the positional indices of 1 and 2. The closing item `"green"` with a positional index of `3` is excluded.

On the other hand, `.loc` includes the closing element:

In [11]:
# Return the elements with the explicit index between 3 and 8
colors.loc[3:8]

3      blue
5     green
8    yellow
dtype: object

This code block says to return all elements with a label index between 3 and 8. Here, the closing item `"yellow"` has a label index of `8` and is included in the output.

You can also pass a negative positional index to `.iloc`:

In [21]:
colors.iloc[-2]

'green'

> **Note:** There used to be an `.ix` indexer, which tried to guess whether it should apply positional or label indexing depending on the data type of the index. Because it caused a lot of confusion, it has been deprecated since Pandas version 0.20.0.
>
> It’s highly recommended that you **do not use `.ix`** for indexing. Instead, always use .loc for label indexing and .iloc for positional indexing. For further details, check out the [Pandas User Guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated).

You can use the code blocks above to distinguish between two Series behaviors:

- You can use `.iloc` on a Series similar to using `[]` on a list.
- You can use `.loc` on a Series similar to using `[]` on a dictionary.

Be sure to keep these distinctions in mind as you access elements of your `Series` objects.

## Accessing DataFrame Elements

Since a `DataFrame` consists of `Series` objects, you can use the very same tools to access its elements. The crucial difference is the additional dimension of the DataFrame. You’ll use the indexing operator for the columns and the access methods `.loc` and `.iloc` on the rows.

### Using the Indexing Operator

If you think of a `DataFrame` as a dictionary whose values are Series, then it makes sense that you can access its columns with the indexing operator:

In [ ]:
city_revenues = pd.Series(
    [4200, 8000, 6500],
    index=["Amsterdam", "Toronto", "Tokyo"]
)
city_employee_count = pd.Series({"Amsterdam": 5, "Tokyo": 8})

In [16]:
city_data = pd.DataFrame({
    "revenue": city_revenues,
    "employee_count": city_employee_count
})

In [21]:
city_data

revenue  employee_count
Amsterdam     4200             5.0
Tokyo         6500             8.0
Toronto       8000             NaN

In [9]:
city_data["revenue"]

Amsterdam    4200
Tokyo        6500
Toronto      8000
Name: revenue, dtype: int64

In [10]:
type(city_data["revenue"])

pandas.core.series.Series

Here, you use the indexing operator to select the column labeled `"revenue"`.

If the column name is a string, then you can use attribute-style accessing with dot notation as well:

In [50]:
%%timeit
for column in city_data.columns:
    for row in city_data.index:
        a = city_data.loc[row,column]

In [51]:
%%timeit
for column in range(city_data.shape[1]):
    for row in range(city_data.shape[0]):
        print(city_data.iloc[row,column])

4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4200
6500
8000
5.0
8.0
nan
4

KeyboardInterrupt: 

`city_data["revenue"]` and `city_data.revenue` return the same output.

There’s one situation where accessing `DataFrame` elements with dot notation may not work or may lead to surprises. This is when a column name coincides with a DataFrame attribute or method name:

In [44]:
toys = pd.DataFrame([
    {"name": "ball", "shape": "sphere"},
    {"name": "Rubik's cube", "shape": "cube"}
])
toys

name   shape
0          ball  sphere
1  Rubik's cube    cube

In [45]:
toys["shape"]

0    sphere
1      cube
Name: shape, dtype: object

In [14]:
toys.shape

(2, 2)

The indexing operation `toys["shape"]` returns the correct data, but the attribute-style operation `toys.shape` still returns the shape of the `DataFrame`. You should only use attribute-style accessing in interactive sessions or for read operations. You shouldn’t use it for production code or for manipulating data (such as defining new columns).

### Using `.loc` and `.iloc`

Similar to Series, a DataFrame also provides `.loc` and `.iloc` **data access methods**. Remember, `.loc` uses the label and `.iloc` the positional index:

In [52]:
city_data

revenue  employee_count
Amsterdam     4200             5.0
Tokyo         6500             8.0
Toronto       8000             NaN

In [15]:
city_data.loc["Amsterdam"]

revenue           4200.0
employee_count       5.0
Name: Amsterdam, dtype: float64

In [16]:
city_data.loc["Tokyo": "Toronto"]

revenue  employee_count
Tokyo       6500             8.0
Toronto     8000             NaN

In [53]:
city_data.iloc[1]

revenue           6500.0
employee_count       8.0
Name: Tokyo, dtype: float64

In [58]:
city_data.loc['Amsterdam','revenue'] = 4599
city_data

revenue  employee_count
Amsterdam     4599             5.0
Tokyo         6500             8.0
Toronto       8000             NaN

Each line of code selects a different row from `city_data`:

1. `city_data.loc["Amsterdam"]` selects the row with the label index `"Amsterdam"`.
2. `city_data.loc["Tokyo": "Toronto"]` selects the rows with label indices from `"Tokyo"` to `"Toronto"`. Remember, `.loc` is inclusive.
3. `city_data.iloc[1]` selects the row with the positional index `1`, which is `"Tokyo"`.

> **Exercise:** Alright, you’ve used `.loc` and `.iloc` on small data structures. Now, it’s time to practice. Use a data access method to display the second-to-last row of the `city_data` dataset.

> **Solution:** The second-to-last row is the row with the **positional index** of `-2`. You can display it with `.iloc`:
> ```bash
> $ city_data.iloc[-2]
> revenue           6500.0
> employee_count       8.0
> Name: Tokyo, dtype: float64
> ```
> 
> You’ll see the output as a `Series` object.

For a `DataFrame`, the data access methods `.loc` and `.iloc` also accept a second parameter. While the first parameter selects rows based on the indices, the second parameter selects the columns. You can use these parameters together to select a subset of rows and columns from your DataFrame:

In [19]:
city_data.loc["Amsterdam": "Tokyo", "revenue"]

Amsterdam    4200
Tokyo        6500
Name: revenue, dtype: int64

Note that you separate the parameters with a comma (`,`). The first parameter, `"Amsterdam" : "Tokyo,"` says to select all rows between those two labels. The second parameter comes after the comma and says to select the `"revenue"` column.

## Querying Your Dataset

In this tutorial, you’ll analyze NBA results provided by FiveThirtyEight in a 17MB CSV file. Run the following block to download the data:

In [59]:
!pip install requests
import requests

download_url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/nba-elo/nbaallelo.csv"
target_csv_path = "nba_all_elo.csv"

response = requests.get(download_url)
response.raise_for_status()    # Check that the request was successful
with open(target_csv_path, "wb") as f:
    f.write(response.content)
print("Download ready.")

Download ready.


When you execute the previous block of code, it will save the file `nba_all_elo.csv` in your current working directory.

> **Note:** You could also use your web browser to download the CSV file.
> 
> However, having a download script has several advantages:
> 
> - You can tell where you got your data.
> - You can repeat the download anytime! That’s especially handy if the data is often refreshed.
> - You don’t need to share the 17MB CSV file with your co-workers. Usually, it’s enough to share the download script.

Now you can use the Pandas Python library to take a look at your data:

In [1]:
import pandas as pd
nba = pd.read_csv("nba_all_elo.csv")
type(nba)

pandas.core.frame.DataFrame

In [3]:
nba.head()

gameorder       game_id lg_id  _iscopy  year_id  date_game  seasongame  \
0          1  194611010TRH   NBA        0     1947  11/1/1946           1   
1          1  194611010TRH   NBA        1     1947  11/1/1946           1   
2          2  194611020CHS   NBA        0     1947  11/2/1946           1   
3          2  194611020CHS   NBA        1     1947  11/2/1946           2   
4          3  194611020DTF   NBA        0     1947  11/2/1946           1   

   is_playoffs team_id  fran_id  ...  win_equiv  opp_id  opp_fran  opp_pts  \
0            0     TRH  Huskies  ...  40.294830     NYK    Knicks       68   
1            0     NYK   Knicks  ...  41.705170     TRH   Huskies       66   
2            0     CHS    Stags  ...  42.012257     NYK    Knicks       47   
3            0     NYK   Knicks  ...  40.692783     CHS     Stags       63   
4            0     DTF  Falcons  ...  38.864048     WSC  Capitols       50   

   opp_elo_i  opp_elo_n  game_location  game_result  forecast notes  
0  1300.0000  1306.7233              H            L  0.640065   NaN  
1  1300.0000  1293.2767              A            W  0.359935   NaN  
2  1306.7233  1297.0712              H            W  0.631101   NaN  
3  1300.0000  1309.6521              A            L  0.368899   NaN  
4  1300.0000  1320.3811              H            L  0.640065   NaN  

[5 rows x 23 columns]

Here, you follow the convention of importing Pandas in Python with the `pd` alias. Then, you use `.read_csv()` to read in your dataset and store it as a DataFrame object in the variable `nba`.

> **Note:** You will learn how to read and write dataframe and series files with pandas later.

You’ve seen how to access subsets of a huge dataset based on its indices. Now, you’ll select rows based on the values in your dataset’s columns to **query** your data. For example, you can create a new `DataFrame` that contains only games played after 2010:

In [4]:
nba.shape

(126314, 23)

In [71]:
current_decade = nba[nba["year_id"] > 2010]

In [72]:
current_decade.shape

(12658, 23)

You now have 24 columns, but your new DataFrame only consists of rows where the value in the `"year_id"` column is greater than `2010`.

You can also select the rows where a specific field is not null:

BOTH NOT NUL AND NOTNA IS THE SAME :TODO https://datascience.stackexchange.com/questions/37878/difference-between-isna-and-isnull-in-pandas

In [8]:
nba[nba["notes"].notnull()].head()

gameorder       game_id lg_id  _iscopy  year_id  date_game  seasongame  \
930         466  194801300BLB   NBA        1     1948  1/30/1948          28   
931         466  194801300BLB   NBA        0     1948  1/30/1948          31   
1084        543  194803230CHS   NBA        1     1948  3/23/1948          49   
1085        543  194803230CHS   NBA        0     1948  3/23/1948          49   
1088        545  194803250CHS   NBA        1     1948  3/25/1948          49   

      is_playoffs team_id    fran_id  ...  win_equiv  opp_id   opp_fran  \
930             0     WSC   Capitols  ...  47.065742     BLB  Baltimore   
931             0     BLB  Baltimore  ...  46.407490     WSC   Capitols   
1084            1     WSC   Capitols  ...  47.710297     CHS      Stags   
1085            1     CHS      Stags  ...  46.089687     WSC   Capitols   
1088            1     BLB  Baltimore  ...  50.044666     CHS      Stags   

      opp_pts  opp_elo_i  opp_elo_n  game_location  game_result  forecast  \
930        95  1434.0548  1448.5005              A            L  0.407940   
931        71  1469.3469  1454.9012              H            W  0.592060   
1084       74  1440.7186  1445.4203              A            L  0.393962   
1085       70  1465.9006  1461.1990              H            W  0.606038   
1088       72  1445.4203  1438.3618              A            W  0.403142   

                                     notes  
930   at Baltimore's Fifth Regiment Armory  
931   at Baltimore's Fifth Regiment Armory  
1084                            Tiebreaker  
1085                            Tiebreaker  
1088                            Tiebreaker  

[5 rows x 23 columns]

In [74]:
games_with_notes = nba[nba["notes"].notnull()]

In [75]:
games_with_notes.shape

(5424, 23)

This can be helpful if you want to avoid any missing values in a column. You can also use `.notna()` to achieve the same goal.

In [76]:
games_with_notes = nba[nba["notes"].notna()]

In [77]:
games_with_notes.shape

(5424, 23)

You can even access values of the object data type as `str` and perform string methods on them:

In [13]:
nba.head(1)

gameorder       game_id lg_id  _iscopy  year_id  date_game  seasongame  \
0          1  194611010TRH   NBA        0     1947  11/1/1946           1   

   is_playoffs team_id  fran_id  ...  win_equiv  opp_id  opp_fran  opp_pts  \
0            0     TRH  Huskies  ...   40.29483     NYK    Knicks       68   

  opp_elo_i  opp_elo_n  game_location  game_result  forecast notes  
0    1300.0  1306.7233              H            L  0.640065   NaN  

[1 rows x 23 columns]

In [20]:
for ind in nba.index:
    
    if nba.loc[ind,'fran_id'].endswith('ers'):
        print(nba.loc[ind,'fran_id'])
        break

Steamrollers


In [19]:
ers = nba[nba["fran_id"].str.endswith("ers")]
ers.head()

gameorder       game_id lg_id  _iscopy  year_id  date_game  seasongame  \
7           4  194611020PRO   NBA        0     1947  11/2/1946           1   
9           5  194611020STB   NBA        0     1947  11/2/1946           1   
16          9  194611050DTF   NBA        1     1947  11/5/1946           2   
20         11  194611070PRO   NBA        0     1947  11/7/1946           2   
22         12  194611070STB   NBA        0     1947  11/7/1946           3   

    is_playoffs team_id       fran_id  ...  win_equiv  opp_id  opp_fran  \
7             0     PRO  Steamrollers  ...  41.540619     BOS   Celtics   
9             0     STB       Bombers  ...  41.492020     PIT   Ironmen   
16            0     STB       Bombers  ...  42.347137     DTF   Falcons   
20            0     PRO  Steamrollers  ...  42.206284     CHS     Stags   
22            0     STB       Bombers  ...  41.272594     NYK    Knicks   

    opp_pts  opp_elo_i  opp_elo_n  game_location  game_result  forecast notes  
7        53  1300.0000  1294.8458              H            W  0.640065   NaN  
9        51  1300.0000  1295.3092              H            W  0.640065   NaN  
16       49  1279.6189  1271.4624              A            W  0.393811   NaN  
20       65  1316.0840  1309.7350              H            W  0.625446   NaN  
22       68  1297.0712  1307.3197              H            L  0.660710   NaN  

[5 rows x 23 columns]

In [32]:
ers.shape

(27797, 23)

You use `.str.endswith()` to filter your dataset and find all games where the home team’s name ends with `"ers"`.

You can combine multiple criteria and query your dataset as well. To do this, be sure to put each one in parentheses and use the logical operators `|` and `&` to separate them.

> **Note:** The operators `and`, `or`, `&&`, and `||` won’t work here. If you’re curious as to why, then check out the section on how the Pandas Python library uses Boolean operators in Python Pandas: **Tricks & Features You May Not Know**.

Do a search for Baltimore games where both teams scored over 100 points. In order to see each game only once, you’ll need to exclude duplicates:

In [23]:
# import numpy as np
# a = np.array([1,2,3,4,5,6,7,8])
x = 4
if x == 2 or x ==3:
    print(x)

In [35]:
nba[
    (nba["_iscopy"] == 0) &
    (nba["pts"] > 100) &
    (nba["opp_pts"] > 100) &
    (nba["team_id"] == "BLB")
]

gameorder       game_id lg_id  _iscopy  year_id  date_game  seasongame  \
1726        864  194902260BLB   NBA        0     1949  2/26/1949          53   
4890       2446  195301100BLB   NBA        0     1953  1/10/1953          32   
4909       2455  195301140BLB   NBA        0     1953  1/14/1953          34   
5208       2605  195303110BLB   NBA        0     1953  3/11/1953          66   
5825       2913  195402220BLB   NBA        0     1954  2/22/1954          60   

      is_playoffs team_id    fran_id  ...  win_equiv  opp_id  opp_fran  \
1726            0     BLB  Baltimore  ...  38.557545     MNL    Lakers   
4890            0     BLB  Baltimore  ...  25.797792     BOS   Celtics   
4909            0     BLB  Baltimore  ...  24.876236     MNL    Lakers   
5208            0     BLB  Baltimore  ...  19.579676     NYK    Knicks   
5825            0     BLB  Baltimore  ...  20.736986     BOS   Celtics   

      opp_pts  opp_elo_i  opp_elo_n  game_location  game_result  forecast  \
1726      115  1637.9852  1640.4923              H            L  0.338936   
4890      105  1591.1434  1563.1652              H            W  0.281855   
4909      112  1665.4396  1668.9125              H            L  0.224238   
5208      113  1649.1516  1651.4359              H            L  0.178973   
5825      111  1591.4943  1593.2749              H            L  0.253365   

                notes  
1726              NaN  
4890              NaN  
4909              NaN  
5208     at Boston MA  
5825  at Worcester MA  

[5 rows x 23 columns]

Here, you use `nba["_iscopy"] == 0` to include only the entries that aren’t copies.

> **Exercise:** Try to build another query with multiple criteria. In the spring of 1992, both teams from Los Angeles had to play a home game at another court. Query your dataset to find those two games. Both teams have an ID starting with `"LA"`.

> **Solution:** You can use `.str` to find the team IDs that start with `"LA"`, and you can assume that such an unusual game would have some notes:
>
> ```python
nba[
    (nba["_iscopy"] == 0) &
    (nba["team_id"].str.startswith("LA")) &
    (nba["year_id"]==1992) &
    (nba["notes"].notnull())
]
> ```
> 
> Your output should show two games on the day `5/3/1992`.

When you know how to query your dataset with multiple criteria, you’ll be able to answer more specific questions about your dataset.